In [71]:
from playsound import playsound
import pandas as pd

data = pd.read_csv('../preprocess/202111171547_all.csv')
df = pd.DataFrame(data)
df = df.dropna()
data = df.iloc[:, :-1]
target = df['target']

In [72]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, stratify=target, random_state=0)
train_input, val_input, train_target, val_target = train_test_split(train_input, train_target, test_size=0.2, stratify=train_target, random_state=0)

In [73]:
from tensorflow import keras
import tensorflow.keras.backend as K

K.clear_session()
model = keras.Sequential()
model.add(keras.layers.LSTM(32, activation="tanh", input_shape=(train_input.shape[1],1), return_sequences=True))
model.add(keras.layers.LSTM(16, activation="tanh", dropout=0.3))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 160, 32)           4352      
                                                                 
 lstm_1 (LSTM)               (None, 16)                3136      
                                                                 
 dense (Dense)               (None, 1)                 17        
                                                                 
Total params: 7,505
Trainable params: 7,505
Non-trainable params: 0
_________________________________________________________________


In [74]:
model.compile(optimizer='rmsprop', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

checkpoint_cb = keras.callbacks.ModelCheckpoint('11241718.h5')
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 10, restore_best_weights=True)

history = model.fit(train_input, train_target, 
                    epochs=100, batch_size = 50,
                    validation_data = (val_input, val_target),
                    callbacks=[checkpoint_cb, early_stopping_cb])

playsound('audio.mp3')

Epoch 1/100
64/64 [==============================] - 4s 27ms/step - loss: 0.5029 - accuracy: 0.7709 - val_loss: 0.5296 - val_accuracy: 0.7799
Epoch 2/100
64/64 [==============================] - 1s 17ms/step - loss: 0.3850 - accuracy: 0.8527 - val_loss: 0.3761 - val_accuracy: 0.8528
Epoch 3/100
64/64 [==============================] - 1s 17ms/step - loss: 0.3635 - accuracy: 0.8615 - val_loss: 0.4360 - val_accuracy: 0.8264
Epoch 4/100
64/64 [==============================] - 1s 17ms/step - loss: 0.3627 - accuracy: 0.8615 - val_loss: 0.3627 - val_accuracy: 0.8616
Epoch 5/100
64/64 [==============================] - 1s 17ms/step - loss: 0.3504 - accuracy: 0.8716 - val_loss: 0.3616 - val_accuracy: 0.8704
Epoch 6/100
64/64 [==============================] - 1s 17ms/step - loss: 0.3496 - accuracy: 0.8678 - val_loss: 0.3628 - val_accuracy: 0.8755
Epoch 7/100
64/64 [==============================] - 1s 17ms/step - loss: 0.3464 - accuracy: 0.8763 - val_loss: 0.3920 - val_accuracy: 0.8566
Epoch 

In [75]:
import datetime

start_time = datetime.datetime.now()
model.evaluate(val_input, val_target)
end_time = datetime.datetime.now()

elapsed_time = end_time - start_time
micro_elapsed_time = elapsed_time.microseconds / 1000
print(micro_elapsed_time)

25/25 [==============================] - 0s 10ms/step - loss: 0.2980 - accuracy: 0.8855
308.448


In [76]:
import datetime

start_time = datetime.datetime.now()
model.evaluate(test_input, test_target)
end_time = datetime.datetime.now()

elapsed_time = end_time - start_time
micro_elapsed_time = elapsed_time.microseconds / 1000
print(micro_elapsed_time)

32/32 [==============================] - 0s 8ms/step - loss: 0.2656 - accuracy: 0.9034
311.062
